### European Soccer Data Analysis.

#### Data Source: https://www.kaggle.com/hugomathien/soccer

#### First we need to import and load the package *dplyr*

In [40]:
install.packages("dplyr", repos="https://cran.r-project.org")

Installing package into 'C:/Users/Haril Satra/Documents/R/win-library/3.3'
(as 'lib' is unspecified)
Warning message:
"package 'dplyr' is in use and will not be installed"

In [41]:
library(dplyr)

#### Connect to the sqlite database using the function src_sqlite

In [42]:
eu_soccer <- src_sqlite("database.sqlite", create = T)

#### Cache all the tables of the database in a standard location

In [43]:
LeagueSQL <- tbl(eu_soccer, sql("SELECT * FROM League"))
CountrySQL <- tbl(eu_soccer, sql("SELECT * FROM Country"))
MatchSQL <- tbl(eu_soccer, sql("SELECT * FROM Match"))
PlayerSQL <- tbl(eu_soccer, sql("SELECT * FROM Player"))
Player_AttributesSQL <- tbl(eu_soccer, sql("SELECT * FROM Player_Attributes"))
TeamSQL <- tbl(eu_soccer, sql("SELECT * FROM Team"))
Team_AttributesSQL <- tbl(eu_soccer, sql("SELECT * FROM Team_Attributes"))

#### Pull down all the above results using the function *collect()*  which will return a tbl_df() of the corresponding input

In [44]:
LeagueDF <- collect(LeagueSQL, n=Inf)
CountryDF <- collect(CountrySQL, n=Inf)
MatchDF <- collect(MatchSQL, n=Inf)
PlayerDF <- collect(PlayerSQL, n=Inf)
Player_AttributesDF <- collect(Player_AttributesSQL, n=Inf)
TeamDF <- collect(TeamSQL, n=Inf)
Team_AttributesDF <- collect(Team_AttributesSQL, n=Inf)

#### Merge the 'player' and 'player_attributes' table with only the columns that we need

In [45]:
PlayerMerge = merge(PlayerDF[,c("player_api_id","player_name")],Player_AttributesDF[,c("player_api_id","player_fifa_api_id","overall_rating","gk_diving", "gk_handling", "gk_kicking", "gk_positioning","gk_reflexes")],by="player_api_id")

In [46]:
head(PlayerMerge)

player_api_id,player_name,player_fifa_api_id,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
2625,"Patryk Rachwal,18",148544,61,12,11,6,8,8
2625,"Patryk Rachwal,18",148544,61,12,11,6,8,8
2625,"Patryk Rachwal,18",148544,61,12,11,6,8,8
2625,"Patryk Rachwal,18",148544,61,12,11,6,8,8
2625,"Patryk Rachwal,18",148544,61,12,11,6,8,8
2625,"Patryk Rachwal,18",148544,58,12,11,6,8,8


#### Summing up all the gk_attribute columns into one single column *gk_overall* which will represent the *Overall rating of the goalkeeper* using the mutate function.

In [47]:
PlayerMerge <- mutate(PlayerMerge,gk_overall = gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes)

#### Removing all the columns which are not needed now like all the gk_attributes except the gk_overall columns

In [48]:
PlayerMerge$gk_diving <- NULL
PlayerMerge$gk_handling <- NULL
PlayerMerge$gk_kicking <- NULL
PlayerMerge$gk_positioning <- NULL
PlayerMerge$gk_reflexes <- NULL

#### Removing all the columns where either the overall_rating or the gk_overall is NA

In [49]:
PlayerMerge <- PlayerMerge[!(is.na(PlayerMerge$overall_rating) | is.na(PlayerMerge$gk_overall)), ]
head(PlayerMerge)

player_api_id,player_name,player_fifa_api_id,overall_rating,gk_overall
2625,"Patryk Rachwal,18",148544,61,45
2625,"Patryk Rachwal,18",148544,61,45
2625,"Patryk Rachwal,18",148544,61,45
2625,"Patryk Rachwal,18",148544,61,45
2625,"Patryk Rachwal,18",148544,61,45
2625,"Patryk Rachwal,18",148544,58,45


#### Getting all the columns home_player_* and away_player_* into a data frame and passing it to a table so that it gives the frequency of each player id which will in turn correspond to the number of appearances of that player.

In [50]:
PlayerMatches <- data.frame(Matches=c(MatchDF$home_player_1,MatchDF$home_player_2,MatchDF$home_player_3,MatchDF$home_player_4,MatchDF$home_player_5,MatchDF$home_player_6,MatchDF$home_player_7,MatchDF$home_player_8,MatchDF$home_player_9,MatchDF$home_player_10,MatchDF$home_player_11,MatchDF$away_player_1,MatchDF$away_player_2,MatchDF$away_player_3,MatchDF$away_player_4,MatchDF$away_player_5,MatchDF$away_player_6,MatchDF$away_player_7,MatchDF$away_player_8,MatchDF$away_player_9,MatchDF$away_player_10,MatchDF$away_player_11))

In [51]:
PlayerMatches <- as.data.frame(table(PlayerMatches))

In [52]:
colnames(PlayerMatches) <- c("player_api_id","No_of_matches")
head(PlayerMatches)

player_api_id,No_of_matches
2625,90
2752,81
2768,71
2770,16
2790,50
2796,1


#### Merge the No_of_matches column that we got above with the data frame we made for gk_overall

In [53]:
temp <- merge(PlayerMerge,PlayerMatches,by="player_api_id")
head(temp)

player_api_id,player_name,player_fifa_api_id,overall_rating,gk_overall,No_of_matches
2625,"Patryk Rachwal,18",148544,61,45,90
2625,"Patryk Rachwal,18",148544,61,45,90
2625,"Patryk Rachwal,18",148544,61,45,90
2625,"Patryk Rachwal,18",148544,61,45,90
2625,"Patryk Rachwal,18",148544,61,45,90
2625,"Patryk Rachwal,18",148544,58,45,90


#### Getting all the columns home_player_* and away_player_*  as one column along with its league id as another column into a data frame and After this step, passing the 

In [54]:
PlayerLeagues <- data.frame(Matches=c(MatchDF$home_player_1,MatchDF$home_player_2,MatchDF$home_player_3,MatchDF$home_player_4,MatchDF$home_player_5,MatchDF$home_player_6,MatchDF$home_player_7,MatchDF$home_player_8,MatchDF$home_player_9,MatchDF$home_player_10,MatchDF$home_player_11,MatchDF$away_player_1,MatchDF$away_player_2,MatchDF$away_player_3,MatchDF$away_player_4,MatchDF$away_player_5,MatchDF$away_player_6,MatchDF$away_player_7,MatchDF$away_player_8,MatchDF$away_player_9,MatchDF$away_player_10,MatchDF$away_player_11),League = MatchDF$league_id)
head(PlayerLeagues)

Matches,League
NA,1
NA,1
NA,1
NA,1
NA,1
NA,1


#### Passing the above generated data frame to a table, gives the frequency of each (player id,league id) which will in turn correspond to the number of appearances of that player in that particular League.

In [55]:
PlayerLeagues <- as.data.frame(table(PlayerLeagues))
head(PlayerLeagues)

Matches,League,Freq
2625,1,0
2752,1,0
2768,1,0
2770,1,14
2790,1,0
2796,1,0


#### Remove all the rows where the frequeny of (player id,league id) is equal to 0 because it means that that player has not played a single game in that league.

In [56]:
PlayerLeagues <- PlayerLeagues[!(PlayerLeagues$Freq==0), ]
head(PlayerLeagues)

,Matches,League,Freq
4,2770,1,14
21,3329,1,85
28,4863,1,25
32,5016,1,69
44,5272,1,27
47,5468,1,7


#### Passing matches column of the above obtained data frame in a table will result in a new column which will correspond to the number of leagues of that player id.

In [57]:
No_of_leagues <- as.data.frame(table(PlayerLeagues$Matches))
head(No_of_leagues)

Var1,Freq
2625,1
2752,1
2768,1
2770,2
2790,1
2796,1


In [58]:
colnames(No_of_leagues) <- c("player_api_id","No_of_Leagues")

#### Merge the No_of_leagues column that we got above with the data frame we made merging N0_of_matches

In [59]:
PlayerMerge <- merge(temp,No_of_leagues,by="player_api_id")
head(PlayerMerge)

player_api_id,player_name,player_fifa_api_id,overall_rating,gk_overall,No_of_matches,No_of_Leagues
2625,"Patryk Rachwal,18",148544,61,45,90,1
2625,"Patryk Rachwal,18",148544,61,45,90,1
2625,"Patryk Rachwal,18",148544,61,45,90,1
2625,"Patryk Rachwal,18",148544,61,45,90,1
2625,"Patryk Rachwal,18",148544,61,45,90,1
2625,"Patryk Rachwal,18",148544,58,45,90,1


#### This above data frame is the final data frame which I will use to answer the questions and hence I save it in a csv using the function write.csv 

In [60]:
write.csv(PlayerMerge,file="player.csv")

#### Merge the 'team' and 'team_attributes' table with only the columns that we need

In [61]:
TeamMerge <- merge(TeamDF[,c("team_api_id","team_fifa_api_id","team_long_name")],Team_AttributesDF[,c("team_api_id","buildUpPlaySpeed","buildUpPlayDribbling","buildUpPlayPassing","chanceCreationPassing","chanceCreationCrossing","chanceCreationShooting","defencePressure","defenceAggression","defenceTeamWidth")],by="team_api_id")

#### Summing up all the numeric buildUpPlay columns into one single column *buildUpPlay* which will represent the *Build Up Play of the team* using the mutate function.

In [62]:
TeamMerge <- mutate(TeamMerge,buildUpPlay = buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing)

#### Summing up all the numeric chanceCreation columns into one single column *chanceCreation* which will represent the *Chance Creation of the team* using the mutate function.

In [63]:
TeamMerge <- mutate(TeamMerge,chanceCreation = chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting)

#### Summing up all the numeric defence columns into one single column *Defence* which will represent the *Defence of the team* using the mutate function.

In [64]:
TeamMerge <- mutate(TeamMerge,defence = defencePressure+defenceAggression+defenceTeamWidth)

#### Removing all the columns which are not needed now like all the individual buildUpPlay, chanceCreation and defence attribute coumns

In [65]:
TeamMerge$buildUpPlaySpeed <- NULL
TeamMerge$buildUpPlayDribbling <- NULL
TeamMerge$buildUpPlayPassing <- NULL
TeamMerge$chanceCreationPassing <- NULL
TeamMerge$chanceCreationCrossing <- NULL
TeamMerge$chanceCreationShooting <- NULL
TeamMerge$defencePressure <- NULL
TeamMerge$defenceAggression <- NULL
TeamMerge$defenceTeamWidth <- NULL

In [66]:
head(TeamMerge)

team_api_id,team_fifa_api_id,team_long_name,buildUpPlay,chanceCreation,defence
1601,874,Ruch Chorzów,NA,155,175
1601,874,Ruch Chorzów,NA,159,136
1601,874,Ruch Chorzów,NA,186,144
1601,874,Ruch Chorzów,139,181,136
1601,874,Ruch Chorzów,NA,159,136
1601,874,Ruch Chorzów,133,176,136


#### Removing all the NA in the data frame and replace it with 0

In [67]:
#Replace NA with 0
TeamMerge[is.na(TeamMerge)] <- 0
head(TeamMerge)

team_api_id,team_fifa_api_id,team_long_name,buildUpPlay,chanceCreation,defence
1601,874,Ruch Chorzów,0,155,175
1601,874,Ruch Chorzów,0,159,136
1601,874,Ruch Chorzów,0,186,144
1601,874,Ruch Chorzów,139,181,136
1601,874,Ruch Chorzów,0,159,136
1601,874,Ruch Chorzów,133,176,136


#### Save the home team's id in one column and corresponding goals in another column from the match table and save it to a data frame. Do the same for the Away team.

In [68]:
Matches_Home <- data.frame(team_api_id=MatchDF$home_team_api_id,Goals=MatchDF$home_team_goal)
Matches_Away <- data.frame(team_api_id=MatchDF$away_team_api_id,Goals=MatchDF$away_team_goal)

#### Merge the above two data frames row wise and save it into another data frame. So now we have all the teams and the goals they scored in each match they played

In [69]:
Home_Away <- rbind(Matches_Home,Matches_Away)
head(Home_Away)

team_api_id,Goals
9987,1
10000,0
9984,0
9991,5
7947,1
8203,1


#### Sum the goals for each of the team using the group_by and the sum function. This will result in two columns one containing the team id and the other column contains the total number of goals it has scored.

In [70]:
grp <- group_by(Home_Away, team_api_id)
Goals <- summarise(grp, sum=sum(Goals))
colnames(Goals) <- c("team_api_id","goals")
head(Goals)

team_api_id,goals
1601,280
1773,126
1957,287
2033,138
2182,372
2183,184


#### Merge the above generated data frame with our main team data frame on the basis of the team_id.

In [71]:
TeamMerge <- merge(TeamMerge,Goals,by="team_api_id")
head(TeamMerge)

team_api_id,team_fifa_api_id,team_long_name,buildUpPlay,chanceCreation,defence,goals
1601,874,Ruch Chorzów,0,155,175,280
1601,874,Ruch Chorzów,0,159,136,280
1601,874,Ruch Chorzów,0,186,144,280
1601,874,Ruch Chorzów,139,181,136,280
1601,874,Ruch Chorzów,0,159,136,280
1601,874,Ruch Chorzów,133,176,136,280


#### This above data frame is the final data frame which I will use to answer the questions and hence I save it in a csv using the function write.csv

In [72]:
write.csv(TeamMerge,file="team.csv")

#### I read the csv file of player which I created earlier and which contains the player information we need to answer the queries.

In [73]:
players <- read.csv("player.csv")

#### Which are the top 10 players by overall rating?

In [74]:
Overall <- summarise(group_by(players, player_api_id,player_name), overall_rating=max(overall_rating))
head(arrange(Overall,desc(overall_rating)),n=10)

player_api_id,player_name,overall_rating
30981,Lionel Messi,94
30717,Gianluigi Buffon,93
30829,Wayne Rooney,93
30893,Cristiano Ronaldo,93
39854,Xavi Hernandez,92
39989,Gregory Coupet,92
30626,Thierry Henry,91
30627,John Terry,91
30657,Iker Casillas,91
30723,Alessandro Nesta,91


#### Which are the top 10 goalkeepers by sum of gk attributes?

In [75]:
Goalkeeper <- summarise(group_by(players, player_api_id,player_name), gk_overall=max(gk_overall))
head(arrange(Goalkeeper,desc(gk_overall)),n=10)

player_api_id,player_name,gk_overall
30717,Gianluigi Buffon,449
39989,Gregory Coupet,447
30859,Petr Cech,445
30657,Iker Casillas,442
27299,Manuel Neuer,440
30989,Julio Cesar,438
24503,Sebastian Frey,437
30726,Edwin van der Sar,436
182917,David De Gea,429
30660,Pepe Reina,428


#### Which are the top 10 players by number of appearances (with any team they played with)?

In [76]:
Appearances <- summarise(group_by(players, player_api_id,player_name), Appearances=max(No_of_matches))
head(arrange(Appearances,desc(Appearances)),n=10)

player_api_id,player_name,Appearances
31293,Steve Mandanda,300
41097,Stephane Ruffier,294
33764,Gorka Iraizoz Moreno,286
42422,Samir Handanovic,286
26295,Hugo Lloris,282
31465,Tim Howard,282
31432,Joe Hart,275
26168,Cedric Carrasso,268
26399,Daniel Congre,262
94550,Moussa Sissoko,260


#### Which are the top 10 players by number of leagues they played in?

In [77]:
Leagues <- summarise(group_by(players, player_api_id,player_name), No_of_Leagues=max(No_of_Leagues))
head(arrange(Leagues,desc(No_of_Leagues)),n=10)

player_api_id,player_name,No_of_Leagues
42119,Gelson Fernandes,6
116750,Ricky van Wolfswinkel,5
178538,Filip Djuricic,5
22543,Radamel Falcao,4
24123,Jonathan De Guzman,4
25816,Almen Abdi,4
26434,Ibrahim Afellay,4
26489,Edson Braafheid,4
26675,Eljero Elia,4
30477,Danijel Pranjic,4


#### I read the csv file of team which I created earlier and which contains the team information we need to answer the queries.

In [78]:
team <- read.csv("team.csv")

#### Which are the top 10 teams by sum of build up play attributes?

In [79]:
Build_Up_Play <- summarise(group_by(team, team_api_id,team_long_name), buildUpPlay=max(buildUpPlay))
head(arrange(Build_Up_Play,desc(buildUpPlay)),n=10)

team_api_id,team_long_name,buildUpPlay
9804,Torino,209
8600,Udinese,205
8194,Fortuna Düsseldorf,198
8033,Podbeskidzie Bielsko-Biala,193
8245,Korona Kielce,192
8024,Widzew Lódz,190
8244,Widzew Lódz,190
208931,Carpi,190
9830,FC Nantes,189
7943,Sassuolo,188


#### Which are the top 10 teams by sum of chance creation attributes?

In [80]:
Chance_Creation <- summarise(group_by(team, team_api_id,team_long_name), chanceCreation=max(chanceCreation))
head(arrange(Chance_Creation,desc(chanceCreation)),n=10)

team_api_id,team_long_name,chanceCreation
8543,Lazio,220
8697,SV Werder Bremen,216
9875,Napoli,215
8178,Bayer 04 Leverkusen,210
8191,Burnley,210
8344,Cardiff City,210
8456,Manchester City,210
8485,Aberdeen,210
8548,Rangers,210
8586,Tottenham Hotspur,210


#### Which are the top 10 teams by sum of defense attributes?

In [81]:
Defence <- summarise(group_by(team, team_api_id,team_long_name), defence=max(defence))
head(arrange(Defence,desc(defence)),n=10)

team_api_id,team_long_name,defence
1957,Jagiellonia Bialystok,210
7794,SC Bastia,210
8021,Zaglebie Lubin,210
8197,Leicester City,210
8344,Cardiff City,210
8350,1. FC Kaiserslautern,210
8466,Southampton,210
8481,AS Nancy-Lorraine,210
8483,Blackpool,210
8521,Stade Brestois 29,210


#### Which are the top 10 teams by number of scored goals?

In [82]:
Total_Goals <- summarise(group_by(team, team_api_id,team_long_name), goals=max(goals))
head(arrange(Total_Goals,desc(goals)),n=10)

team_api_id,team_long_name,goals
8634,FC Barcelona,849
8633,Real Madrid CF,843
9925,Celtic,695
9823,FC Bayern Munich,653
8640,PSV,652
8593,Ajax,647
9931,FC Basel,619
8456,Manchester City,606
8455,Chelsea,583
10260,Manchester United,582
